In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,MultinomialNB
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load dataset

In [ ]:
df= pd.read_csv('../input/heart-disease-uci/heart.csv')
df.head()

In [ ]:
print('shape of data:',df.shape)

***Identify caterofical and numerical data columns***

In [ ]:
catgorical_column = [i for i in df.columns if df[i].nunique()<10]
print(catgorical_column)
numerical_column = [i for i in df.columns if df[i].nunique()>10]
print(numerical_column)

**Check null values**

In [ ]:
print('Null values:',df.isna().sum().sum())

# EDA

In [ ]:
target=df['target'].value_counts()

In [ ]:
plt.pie(target,
       autopct='%1.2f%%',
       labels=['0','1'])
plt.legend()
plt.show()

In [ ]:
sns.countplot(df['sex'],)
plt.legend()
plt.show()

In [ ]:
sns.pairplot(df)

In [ ]:
df[['cp','target']].groupby(['cp'],as_index=False).mean()

In [ ]:
df['trestbps_grp']=pd.cut(df['trestbps'],7)
df[['trestbps_grp','target']].groupby(['trestbps_grp'],as_index=False).mean()

In [ ]:
df['chol_grp']=pd.cut(df['chol'],5)
df[['chol_grp','target']].groupby(['chol_grp'],as_index=False).mean()

In [ ]:
df[['fbs','target']].groupby(['fbs'],as_index=False).mean()

In [ ]:
df[['restecg','target']].groupby(['restecg'],as_index=False).mean()

In [ ]:
df['thalach_grp']=pd.cut(df['thalach'],4)
df[['thalach_grp','target']].groupby(['thalach_grp'],as_index=False).mean()

In [ ]:
num =df[['age', 'trestbps', 'chol', 'thalach', 'oldpeak',]]

In [ ]:
sns.swarmplot(x='fbs',y='oldpeak',data=df,hue='target')
plt.show()

In [ ]:
fig,(a1,a2,a3)=plt.subplots(1,3,figsize=(10,5))
sns.boxplot(y=df['age'],ax=a1)
sns.boxplot(y=df['trestbps'],ax=a2)
sns.boxplot(y=df['chol'],ax=a3)
plt.show()
fig,(a1,a2)=plt.subplots(1,2,figsize=(8,5))
sns.boxplot(y=df['thalach'],ax=a1)
sns.boxplot(y=df['oldpeak'],ax=a2)

# Remove outliers

In [ ]:
z=np.abs(stats.zscore(df[numerical_column]))
print(z)
print(np.where(z>3))

In [ ]:
z_price = df[(z<3).all(axis=1)]
df.shape,z_price.shape

In [ ]:
z_price.head()

# Prepare data for traning and testing

In [ ]:
X = df.iloc[:,3:-4]
y = df['target']
mm= StandardScaler()
X=pd.DataFrame(mm.fit_transform(X))


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=5,stratify =y)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

# ML

***KNN***

In [ ]:
wt = {'uniform':[],'distance':[]}
for i in wt.keys():
    for j in range(1,21,2):
        knn = KNeighborsClassifier(n_neighbors=j,weights=i)
        knn.fit(X_train,y_train)
        y_predict=knn.predict(X_test)
        wt[i].append(accuracy_score(y_test,y_predict))

In [ ]:
plt.plot(wt['uniform'],marker='o',c='r')
plt.plot(wt['distance'],marker='o',c='g')
plt.show()

In [ ]:
wt = {'1':[],'2':[]}
for i in wt.keys():
    for j in range(1,21,2):
        knn = KNeighborsClassifier(n_neighbors=j,weights='uniform',p=int(i))
        knn.fit(X_train,y_train)
        y_predict=knn.predict(X_test)
        wt[i].append(accuracy_score(y_test,y_predict))

In [ ]:
plt.plot(wt['1'],marker='o',c='r')
plt.plot(wt['2'],marker='o',c='g')
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=j,weights='uniform',p=1)
knn.fit(X_train,y_train)
y_predict=knn.predict(X_test)
knn.score(X_test,y_test)

In [ ]:
KNN_con=confusion_matrix(y_test,y_predict)

In [ ]:
knn_score = round(knn.score(X_test,y_test)*100)
knn_score

# Decission Tree

In [ ]:
dec = DecisionTreeClassifier(criterion='entropy',min_samples_split=2,max_features=2,max_depth=3)
dec.fit(X_train,y_train)
y_predict = dec.predict(X_test)
print('test',confusion_matrix(y_test,y_predict))
print('test:',dec.score(X_test,y_test))
print('train:',dec.score(X_train,y_train))

dec_con=confusion_matrix(y_test,y_predict)
decission_tree=round(dec.score(X_test,y_test)*100)

In [ ]:
plt.figure(figsize=[10,10])
_ = plot_tree(dec,filled=True,node_ids=True)

# Random-forest

In [ ]:
rnd =RandomForestClassifier(n_estimators=15,criterion='entropy',min_samples_split=2,max_features=2,max_depth=3)
rnd.fit(X_train,y_train)
y_predict = rnd.predict(X_test)
print('train:-',rnd.score(X_train,y_train))
print('test:-',rnd.score(X_test,y_test))
rnd_con=confusion_matrix(y_test,y_predict)

In [ ]:
random_forest= round(rnd.score(X_test,y_test)*100)
random_forest

# Navi Bayes

In [ ]:
guss=GaussianNB()
guss.fit(X_train,y_train)
y_predict = guss.predict(X_test)
print(confusion_matrix(y_test,y_predict))
print(accuracy_score(y_test,y_predict))
gussian=accuracy_score(y_test,y_predict)*100

navi_con=confusion_matrix(y_test,y_predict)

# Comparing model

In [ ]:
model=pd.DataFrame({
    'model':['KNN','Decission Tree','Random forest','Navi bayes'],
    'score':[knn_score,decission_tree,random_forest,gussian]
})

In [ ]:
model.sort_values(by='score',ascending=False)

In [ ]:
sns.barplot(x='model',y='score',data=model)

# Conclusion

In [ ]:
plt.figure(figsize=[10,10])



plt.subplot(2,2,1)
plt.title('KNN')
sns.heatmap(KNN_con,annot=True,cbar=False)

plt.subplot(2,2,2)
plt.title('Decission tree')
sns.heatmap(dec_con,annot=True,cbar=False)

plt.subplot(2,2,3)
sns.heatmap(rnd_con,annot=True,cbar=False)
plt.title('Random Forest',)

plt.subplot(2,2,4)
sns.heatmap(navi_con,annot=True,cbar=False)
plt.title('Navi bayes',)

plt.suptitle('Confusion Matrix')

#  Kindly give feedback and do upvote if you feel its good